<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
from azureml.core.workspace import Workspace
from azureml.core.model import Model
from azureml.core.conda_dependencies import CondaDependencies 
from azureml.core.image import ContainerImage
from azureml.core.webservice import AciWebservice, Webservice, AksWebservice
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.exceptions import ComputeTargetException
from azureml.core.model import InferenceConfig
import os

In [6]:
inference_dir = './inference'

In [3]:
ws = Workspace.from_config()

In [4]:
models = Model.list(ws)

In [19]:
auc=0
for model in models:
    if float(model.tags['AUC']) > auc:
        auc = float(model.tags['AUC'])
        best_model = model
best_model

Model(workspace=Workspace.create(name='Invasive', subscription_id='68bdd703-8837-469c-80bd-bfb35f3b886f', resource_group='ProjectGroup2'), name=InvasiveCNN, id=InvasiveCNN:2, version=2, tags={'AUC': '0.9659936675027756', 'transfer-learning': 'false', 'image_size': '(128, 128)', 'batch_size': '32', 'epochs': '30'}, properties={})

In [27]:
model_info = best_model.tags
model_info['name'] = best_model.name
model_info['version'] = best_model.version

In [28]:
import json
with open(os.path.join(inference_dir,'model_tags.json'), 'w') as file:
    json.dump(model_info, file)

In [96]:
myenv = CondaDependencies()
myenv.add_conda_package("numpy")
#myenv.add_pip_package("opencv-python")
myenv.add_pip_package("tensorflow==2.2.0")
myenv.add_pip_package("pillow")

with open(os.path.join(inference_dir, "Invasive.yml"),"w") as f:
    f.write(myenv.serialize_to_string())

In [97]:
inference_config = InferenceConfig(source_directory=inference_dir,
                                   runtime= "python", 
                                   entry_script='score.py',
                                   conda_file="Invasive.yml")

In [98]:
from azureml.core.webservice import AciWebservice, Webservice, AksWebservice

In [184]:
# Deploy to ACI

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
aci_service_name = 'invasive-species-id-1'
aci_service = Model.deploy(ws, aci_service_name, [best_model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

Running............................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [185]:
print(aci_service.get_logs())

2020-05-15T13:31:38,512796545+00:00 - gunicorn/run 
2020-05-15T13:31:38,513381549+00:00 - iot-server/run 
2020-05-15T13:31:38,515438561+00:00 - rsyslog/run 
2020-05-15T13:31:38,522003500+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_a5f844aedbaa76f68aa8a50c193863c4/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_a5f844aedbaa76f68aa8a50c193863c4/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_a5f844aedbaa76f68aa8a50c193863c4/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_a5f844aedbaa76f68aa8a50c193863c4/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_a5f844aedbaa76f68aa8a50c193863c4/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
EdgeHubC

In [186]:
for webservice_name in ws.webservices:
    print(webservice_name)

invasive-species-id-1


In [187]:
#service = ws.webservices['invasive-species-id-1']

aci_service.update(enable_app_insights=True)

In [188]:
import pandas as pd
train_dir = "./train_dataset"
test_dir = "./test_dataset"
label_filename = os.path.join(train_dir, "train_labels.csv")
labels = pd.read_csv(label_filename, index_col=0)
labels.head(10)

,invasive
name,
1,0
2,0
3,1
4,0
5,1
6,0
7,1
8,1
9,0


In [195]:
import base64
def local_image_to_string(img_path):
    with open(img_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read())
    return(encoded_string)

img_path = './train_dataset/train/1.jpg'
img_str = local_image_to_string(img_path)

input_data = json.dumps({'data': img_str.decode("utf-8")})

In [196]:
predictions = aci_service.run(input_data = input_data)
predictions

'[[0.03900056704878807]]'

In [197]:
# Call the endpoint
import requests
import json
import numpy as np

headers = {'Content-Type':'application/json'}

resp = requests.post(aci_service.scoring_uri, data=input_data, headers=headers)

print("prediction:", resp.text)

prediction: "[[0.03900056704878807]]"


In [ ]:
print(aci_service.get_logs())

In [ ]:
help(aci_service)

In [ ]:
Webservice(ws, aci_service_name).delete()